# JASPAR DB PARSING

-Jaspar SQL Dump was downloaded from http://jaspar.genereg.net/downloads/ .

-Built the database on my computer using HeidiSQL (MySQL style DB)

-Used PyMySQL and Biopython to access my local copy of the database and pull relevant information, which I compiled into a dictionary.

-Saved this dictionary as a JSON file for future use.


In [ ]:
import sys
!{sys.executable} -m pip install mysqlclient pymysql
#anaconda is too good

In [ ]:
from Bio.motifs.jaspar.db import JASPAR5

JASPAR_DB_HOST = "127.0.0.1" #provided it's on your computer
JASPAR_DB_NAME = DB_NAME
JASPAR_DB_USER = "root" 
JASPAR_DB_PASS = DB_PASS
jdb = JASPAR5(
     host=JASPAR_DB_HOST,
     name=JASPAR_DB_NAME,
     user=JASPAR_DB_USER,
     password=JASPAR_DB_PASS
)

motifs=jdb.fetch_motifs()

-Creating the dictionary below, the format is:
{protein_name: {'class':class,'family':family,'species':species,'acc' = UniProt ID(s),'pubmed':reference, 'motif': motif in pfm format}
Thus, each protein has a dictionary containing its attributes as its value, and the proteins are keys in the overall dictionary.

In [ ]:
jaspardict = {}

for motif in motifs:
    jaspardict[motif.name]={'class':motif.tf_class,'family':motif.tf_family,'species':motif.species, 'acc':motif.acc,'pubmed':motif.medline, 'motif':motif.format("pfm")}



In [ ]:
import pprint
pprint.pprint(jaspardict)


In [ ]:
!{sys.executable} -m pip install beautifulsoup4

-The biopython package gives uniprot taxonomical IDs instead of species names, so for future use I'm adding in the species name to the species key for each tf.

In [ ]:
import sys,requests
from bs4 import BeautifulSoup as soup

query_start = 'https://www.uniprot.org/taxonomy/'
species_ids = []
for tf in jaspardict.values():
    if tf['species']:
        spec_id = tf['species'][0]
    if spec_id not in species_ids:
        species_ids.append(spec_id)
id_to_species = {}
for spec_id in species_ids:
    r = requests.get(query_start+spec_id+"/")
    s = soup(r.text,'html.parser')
    s.prettify()
    id_to_species[spec_id]=s.title.string
print (id_to_species)
        


In [ ]:
for tf in jaspardict.values():
    if tf['species']:
        tf['species'] = str(tf['species'])+" "+id_to_species[tf['species'][0]]


In [ ]:
pprint.pprint(jaspardict)

In [ ]:
import json
h = json.dumps(jaspardict)
f= open('jaspar_final.json','w')
f.write(h)
f.close()